Find articles that made it to certain journals and compare key words.

In [8]:
#Need to add parent directoy to sys.path to find 'metadataDB'
import sys
sys.path.append('../')

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from metadataDB.declareDatabase import *


engine = create_engine("sqlite:///../arXiv_metadata.db", echo=False)
Base.metadata.bind = engine
DBsession = sessionmaker(bind=engine)
session = DBsession()

In [50]:
query = session.query(Article).filter(Article.journal_ref.like('Nature%'))
result = query.all()

In [51]:
for item in result:
    print (item.arxiv_id, item.journal_ref, item.)

(u'1311.7111', u'Nature Commun. 4, 2991 (2013)')
(u'1312.4881', u'Nature 510, 376 (2014)')
(u'astro-ph/0003014', u'Nature 405 (2000) 143-149')
(u'astro-ph/0005526', u'Nature 406 (2000) 158')
(u'astro-ph/0102301', u'Nature, 410, 338-340 (2001)')
(u'astro-ph/0111602', u'Nature, Volume 410, Issue 6830, pp. 773-779 (2001)')
(u'astro-ph/0201409', u'Nature 415 (2002) 403-406')
(u'astro-ph/0309495', u'Nature 425 (2003) 264')
(u'astro-ph/0311282', u'Nature 426 (2003) 257-259')
(u'astro-ph/0401037', u'Nature, (2004), 430. pp 181-184')
(u'astro-ph/0404569', u'Nature 428 (2004) 919-921')
(u'astro-ph/0408478', u'Nature 430 (2004) 999-1001')
(u'astro-ph/0409004', u'Nature 430 (2004) 326-329')
(u'astro-ph/0412092', u'Nature 432 (2004) 596')
(u'astro-ph/0412341', u'Nature 434 (2005) 873-876')
(u'astro-ph/0502441', u'Nature 434 (2005) 873-876')
(u'astro-ph/0503021', u'Nature 434 (2005) 871-873')
(u'astro-ph/0509433', u'Nature 437 (2005) 381-384')
(u'astro-ph/0510096', u'Nature 437 (2005) 859-861')
(u'

In [52]:
session.close()